# Preprocessing Script

In [3]:
import os
import sys
import importlib
from importlib import reload 
from dataclasses import dataclass, field, fields
from itertools import compress
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy
import scipy.io as sio
from scipy import signal
from scipy.signal import spectrogram, hann, butter, filtfilt, freqz
from scipy import stats

import pickle
import json
import csv

#mne
import mne_bids
import mne
from mne.time_frequency import tfr_morlet 
from mne.stats import permutation_cluster_test
from mne.preprocessing import ICA, create_ecg_epochs


In [2]:
jennifer_user_path = os.getcwd()
while jennifer_user_path[-14:] != 'jenniferbehnke':
    jennifer_user_path = os.path.dirname(jennifer_user_path)

# directory to this Repository
project_path = os.path.join(jennifer_user_path, 'code', 'Beta_profile_project', 'Beta_profile')
sys.path.append(project_path)

os.chdir(project_path)

import src.beta.utils.find_folders as find_folders
importlib.reload(find_folders)

# import PyPerceive
project_path = find_folders.chdir_repository("Py_Perceive")

from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)

import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as PyPerceive_find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers



# import all functions from the local repository
project_path = find_folders.chdir_repository("Beta_profile")

# utils
import src.beta.utils.find_folders as find_folders
import src.beta.utils.io as io

# tfr
import src.beta.preprocessing.tfr as tfr

# plots
import src.beta.plotting.power_spectra_plots as power_spectra


importlib.reload(find_folders)
importlib.reload(io)
importlib.reload(tfr)
importlib.reload(power_spectra)

<module 'src.beta.plotting.power_spectra_plots' from '/Users/jenniferbehnke/code/Beta_profile_project/Beta_profile/src/beta/plotting/power_spectra_plots.py'>

### Load MNE Objects from PyPerceive

In [14]:
loaded_data = io.load_py_perceive_object(
    sub="019",
    session="fu3m",
    condition="m0s0",
    hemisphere="Right"
)
loaded_data


	### WARNING: NaNs in Metadata Table sub-019 ###
NaNs in: sub019_ses-2021042207162599_run-BrainSense20210422074600.mat
NaNs in: sub019_ses-2021042207162599_run-BrainSense20210422075600.mat
NaNs in: sub019_ses-2021042207162599_run-IS20210422081000.mat
NaNs in: sub019_ses-2021042207162599_run-IS20210422081300.mat
NaNs in: sub019_ses-2021042207162599_run-IS20210422081500.mat
NaNs in: sub019_ses-2021042207162599_run-IS20210422081900.mat
NaNs in: sub019_ses-2021070506275097_run-BrainSense20210705070700.mat
NaNs in: sub019_ses-2021070506275097_run-BrainSense20210705072100.mat
NaNs in: sub019_ses-2021070507365797_run-BrainSense20210705074000.mat
NaNs in: sub019_ses-2021070507365797_run-BrainSense20210705074300.mat
NaNs in: sub019_ses-2021070507365797_run-BrainSense20210705075000.mat
NaNs in: sub019_ses-2021070507365797_run-BrainSense20210705075800.mat
NaNs in: sub019_ses-2021070507365797_run-BrainSense20210705081000.mat
NaNs in: sub019_ses-2021070508345297_run-BrainSense20210705084300.mat
Na

PerceiveData(sub='019', incl_modalities=['streaming'], incl_session=['fu3m'], incl_condition=['m0s0'], incl_task=['rest'], incl_contact=['RingR', 'SegmIntraR', 'SegmInterR'], import_json=False, warn_for_metaNaNs=True, use_chronic_json_file=True, allow_NaNs_in_metadata=False)

In [33]:
data = io.extract_data_from_py_perceive(
    sub="024",
    session="fu12m",
    condition="m0s0",
    hemisphere="Right"
)
data


	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610105800.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610111500.mat
NaNs in: sub-2021061

{'RingR': <RawArray | 6 x 5288 (21.1 s), ~258 kB, data loaded>,
 'SegmIntraR': <RawArray | 6 x 5288 (21.1 s), ~258 kB, data loaded>,
 'SegmInterR': <RawArray | 3 x 5188 (20.7 s), ~129 kB, data loaded>}

In [38]:
ring_data = data["SegmIntraR"]

In [5]:
structured_signals = tfr.pick_channels_of_interest(
    sub="024",
    session="fu12m",
    condition="m0s0",
    hemisphere="Right"
)


	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610105800.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610111500.mat
NaNs in: sub-2021061

In [6]:
structured_signals

,original_channel,channel,lfp,lfp_group
0,LFP_R_12_STN_MT,12,"[1.3601056882, 2.153500673, 1.7001321103, 2.72...",RingR
1,LFP_R_01_STN_MT,01,"[-4.554200042, -4.668055043, -7.5144300693, -7...",RingR
2,LFP_R_23_STN_MT,23,"[-1.3479618874, -4.6055364487, -6.4028189653, ...",RingR
3,LFP_R_1A1B_STN_MT,1A1B,"[3.4822348759, 4.2685459769, 7.076799909, 7.63...",SegmIntraR
4,LFP_R_1B1C_STN_MT,1B1C,"[-3.6433600336, -2.0493900189, -4.0987800378, ...",SegmIntraR
5,LFP_R_1A1C_STN_MT,1A1C,"[1.2356317301, 2.358933303, 3.5945650331, 3.14...",SegmIntraR
6,LFP_R_2A2B_STN_MT,2A2B,"[0.5667107034, 0.6800528441, -1.2467635475, -2...",SegmIntraR
7,LFP_R_2B2C_STN_MT,2B2C,"[-3.3017950304, -2.6186650241, -2.5048100231, ...",SegmIntraR
8,LFP_R_2A2C_STN_MT,2A2C,"[-4.4932062914, -3.8192253477, -3.145244404, -...",SegmIntraR
9,LFP_R_1A2A_STN_MT,1A2A,"[-25.8359361757, -27.2962282204, -20.444088626...",SegmInterR


### Filter and FT of LFP

In [29]:
beta_profile = tfr.main_tfr(
    sub="024",
    session="fu12m",
    condition="m0s0",
    hemisphere="Right"
)


	### WARNING: NaNs in Metadata Table sub-024 ###
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618063700.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064000.mat
NaNs in: sub024_ses-2021061806255999_run-BrainSense20210618064200.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084000.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084300.mat
NaNs in: sub024_ses-2021061808253999_run-BrainSense20210618084700.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921070500.mat
NaNs in: sub024_ses-2021092106385396_run-BrainSense20210921072300.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921074800.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921080600.mat
NaNs in: sub024_ses-2021092107452096_run-BrainSense20210921082400.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610105800.mat
NaNs in: sub-20210615PStn_ses-2022061010445782_run-BrainSense20220610111500.mat
NaNs in: sub-2021061

In [30]:
# freq range info: peak CF, peak power, average in f band
beta_profile[0]

,channel,f_range,power_in_f_range,peak_CF,peak_power,peak_4Hz_power
0,01,low_beta,0.439947,16.0,0.510512,0.464534
1,01,high_beta,0.187556,23.0,0.326573,0.261101
2,01,beta,0.275344,16.0,0.510512,0.464534
3,12,low_beta,0.199326,16.0,0.251736,0.214283
4,12,high_beta,0.089374,21.0,0.182753,0.158925
5,12,beta,0.127618,16.0,0.251736,0.214283
6,23,low_beta,0.198267,18.0,0.226481,0.199406
7,23,high_beta,0.089804,21.0,0.188326,0.167013
8,23,beta,0.127530,18.0,0.226481,0.199406
9,1A1B,low_beta,0.104303,18.0,0.127099,0.097528


In [31]:
# LFP and PSD of channels
beta_profile[1]

,channel,unfiltered_lfp,filtered_lfp,frequencies,filtered_psd
0,01,"[-4.554200042, -4.668055043, -7.5144300693, -7...","[-0.3055541438904896, -1.0885203035876938, -4....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0050684399327060375, 0.0022647524330400796,..."
1,12,"[1.3601056882, 2.153500673, 1.7001321103, 2.72...","[-0.03508955523947705, 0.7694804077841276, 0.8...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0020488780742291247, 0.0009028849211278945,..."
2,23,"[-1.3479618874, -4.6055364487, -6.4028189653, ...","[-0.00824900657258963, -3.334320975452224, -4....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.002525105266546525, 0.0011038590983910683, ..."
3,1A1B,"[3.4822348759, 4.2685459769, 7.076799909, 7.63...","[-0.04288755968340896, 0.9119064086760256, 3.3...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.00099467844834702, 0.00045162787295239355, ..."
4,1A1C,"[1.2356317301, 2.358933303, 3.5945650331, 3.14...","[0.1882350216478973, 1.0271690260571285, 2.819...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0008318762733432066, 0.00037503102697477783..."
5,1B1C,"[-3.6433600336, -2.0493900189, -4.0987800378, ...","[0.2026947017843661, 1.5454245188805025, -0.49...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.0006305004370323955, 0.00028218996393089806..."
6,2A2B,"[0.5667107034, 0.6800528441, -1.2467635475, -2...","[0.13189252465235402, 0.06745790330573564, -2....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.006461119233177716, 0.0028043416071776395, ..."
7,2A2C,"[-4.4932062914, -3.8192253477, -3.145244404, -...","[0.14191359857741392, 0.7396694967537968, 1.01...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.006070348761739933, 0.002674208290154584, 5..."
8,2B2C,"[-3.3017950304, -2.6186650241, -2.5048100231, ...","[-0.0060997703778540024, 0.9310020358441465, 1...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.006290511408721497, 0.0027299560526191425, ..."
9,1A2A,"[-25.8359361757, -27.2962282204, -20.444088626...","[-0.2551289350008452, -1.7713027380437298, 3.5...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[0.005335302729109903, 0.002376526141481409, 4..."
